# Sentimen Analisis Model Indo

In [ ]:
from gensim.models import Doc2Vec
import re
import os
import pandas as pd
import numpy as np
import multiprocessing
import ipython_genutils
import pickle
import progressbar
import tensorflow as tf
from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPooling1D,GRU,LSTM
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn import utils
from sklearn.utils import shuffle
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

### load file csv

target
1. negative = -1
2. neutral = 0
3. positive = 1

In [ ]:
df = pd.read_csv('lib/dataset',index_col=0)
df1 = pd.read_csv('lib/dataset ahok',index_col=0)
df2 = pd.read_csv('lib/datasetfm',index_col=0)
df=pd.concat([df,df1,df2],sort=False)
#df['target']=df['target'].map({0:-1,1:0,2:1})
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df = shuffle(df)
df.info()
df.head()

### jumlah data tiap class

In [ ]:
df.groupby('target')['text'].count()

### One Hot 

### downsampling dataset

In [ ]:
dneg=df.query('target == -1')
dnet=df.query('target == 0')
dpos=df.query('target == 1')

minlen=min(len(dnet.index),len(dpos.index),len(dneg.index))


dnet=dnet.head(minlen)
dneg=dneg.head(minlen)
dpos=dpos.head(minlen)

data_training=pd.concat([dneg.text,dnet.text,dpos.text])
target=pd.concat([dneg.target,dnet.target,dpos.target])
target=to_categorical(target-target.min())

In [ ]:
# tanpa downsampling
data_training=df.text
target=df.target
target=to_categorical(target-target.min())

In [ ]:
df_ds=pd.DataFrame({'text':data_training,'target':np.argmax(target, axis=1, out=None)})
df_ds.groupby('target')['text'].count()

### tokenizing

create tokenizer lalu di save ada/dibuat tokenizer. jika sudah dibuat maka skip langsung ke load tokenizer

In [ ]:
#save tokenizer
#data=df.text
#tokenizer = Tokenizer(num_words=100000)
#tokenizer.fit_on_texts(data)                        
#sequences = tokenizer.texts_to_sequences(data) 
#with open('lib/tokenizer_indo.pickle', 'wb') as handle:
#    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

load tokenizer lalu konversi dataset jadi vector

In [ ]:
print('tokenizing')
num_words=100000
tokenizer = Tokenizer(num_words)
with open('lib/tokenizer_indo.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
sequences = tokenizer.texts_to_sequences(data_training) 

In [ ]:
sequences[0]

### padding

In [ ]:
print('padding')
input_seq = pad_sequences(sequences, maxlen=70)

In [ ]:
input_seq[0]

### split dataset jadi data training dan data validasi

In [ ]:
input_train, input_val, target_train, target_val = model_selection.train_test_split(input_seq,target,test_size = 0.2, random_state = 0)

print('data train\t: ',len(input_train))
print('data validasi\t: ',len(input_val),)

### Parameter

1. ep = epoch
2. bs = batch size
3. traina = True atau False 
4. lanjut = True atau False

ket.
1. jumlah epoch training model
2. jumlah batch size dlm training
3. True untuk trainable sehingga dalam proses training model word embedding juga di train. False untuk word embedding tidak ikut di train.
4. True untuk me-load model yg sudah di train sebelumnya/pretrained model lalu model tsb di train lagi. False untuk training model dari awal seperti model baru/ blm di train 

In [ ]:
ep=135
bs=32
traina=True
lanjut=False

# Model

Model
1. Word2Vec
2. DBOW
3. DMM
4. DMC
5. DBOW-DMM
6. DBOW-DMC
7. DMM-DMC
8. DBOW-DMM-DMC

### neural network

In [ ]:
def getModel(num_word,embed_dim,embed_weight,input_len,trainable):
    seed = 7
    model = Sequential()
    e = Embedding(num_word, embed_dim, weights=[embed_weight], input_length=input_len, trainable=trainable)
    model.add(e)
    model.add(Conv1D(32,kernel_size=2,padding='same',activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.3))
    model.add(Conv1D(64,kernel_size=2,padding='same',activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.35))
    model.add(Conv1D(128,kernel_size=2,padding='same',activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.4))
    model.add(LSTM(50,return_sequences=True))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    model.summary()
    return model

In [ ]:
def saveModel(model,path):
    model_json = model.to_json()
    with open(path, "w") as json_file:
        json_file.write(model_json)

In [ ]:
def train(model,lanjut, path_weights,input_train,target_train,input_val,target_val,epoch,bs):
    if lanjut==True:
        model.load_weights(path_weights)
    filepath=path_weights
    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    history=model.fit(input_train, target_train,validation_data=(input_val, target_val), shuffle=False, epochs=ep, batch_size=bs,callbacks=callbacks_list)
    return history

In [ ]:
def score(weights,input_val,target_val):
    model.load_weights(weights)
    scores = model.evaluate(input_val, target_val)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print()
    predict = model.predict_classes(input_val)
    y_true = np.argmax(target_val, axis=1, out=None)
    y_pred = predict
    target_names = ['negative', 'neutral', 'positive']
    print (classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
def plot(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

# Train

## Word2Vec

In [ ]:
# load word2vec
model_ug_cbow = KeyedVectors.load('lib/cbow_indo.word2vec')
model_ug_sg = KeyedVectors.load('lib/sg_indo.word2vec')
print('word2vec loaded')

# words embedding
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] =np.append( model_ug_cbow.wv[w],model_ug_sg.wv[w])

num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('embed')

In [ ]:
model=getModel(num_words, 200, embedding_matrix, 70, traina)

In [ ]:
saveModel(model,"lib/model_w2v_indo.json")

In [ ]:
history=train(model,lanjut, 'lib/weights_w2v.hdf5',input_train,target_train,input_val,target_val,ep,bs)

In [ ]:
score('lib/weights_w2v.hdf5',input_val,target_val)

In [ ]:
plot(history)

## DBOW

In [ ]:
model_ug_cbow = Doc2Vec.load('lib/d2v_dbow_indo')

print('word2vec loaded')

# words embedding
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = model_ug_cbow.wv[w]

num_words = 100000
embedding_matrix = np.zeros((num_words, 100))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('embed')

In [ ]:
model=getModel(num_words, 100, embedding_matrix, 70, traina)

In [ ]:
saveModel(model,"lib/model_dbow_indo.json")

In [ ]:
history=train(model,lanjut, 'lib/weights_dbow.hdf5',input_train,target_train,input_val,target_val,ep,bs)

In [ ]:
score('lib/weights_dbow.hdf5',input_val,target_val)

In [ ]:
plot(history)

## DMM

In [ ]:
model_ug_cbow = Doc2Vec.load('lib/d2v_dmm_indo')

print('word2vec loaded')

# words embedding
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = model_ug_cbow.wv[w]

num_words = 100000
embedding_matrix = np.zeros((num_words, 100))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('embed')


In [ ]:
model=getModel(num_words, 100, embedding_matrix, 70, traina)

In [ ]:
saveModel(model,"lib/model_dmm_indo.json")

In [ ]:
history=train(model,lanjut, 'lib/weights_dmm.hdf5',input_train,target_train,input_val,target_val,ep,bs)

In [ ]:
score('lib/weights_dmm.hdf5',input_val,target_val)

In [ ]:
plot(history)

## DMC

In [ ]:
model_ug_cbow = Doc2Vec.load('lib/d2v_dmc_indo')

print('word2vec loaded')

# words embedding
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    #embeddings_index[w] =np.append( model_ug_cbow.wv[w],model_ug_sg.wv[w])
    embeddings_index[w] = model_ug_cbow.wv[w]

num_words = 100000
embedding_matrix = np.zeros((num_words, 100))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('embed')

In [ ]:
model=getModel(num_words, 100, embedding_matrix, 70, traina)

In [ ]:
saveModel(model,"lib/model_dmc_indo.json")

In [ ]:
history=train(model,lanjut, 'lib/weights_dmc.hdf5',input_train,target_train,input_val,target_val,ep,bs)

In [ ]:
score('lib/weights_dmc.hdf5',input_val,target_val)

In [ ]:
plot(history)

## DBOW - DMM

In [ ]:
model_ug_cbow = Doc2Vec.load('lib/d2v_dbow_indo')
model_ug_sg = Doc2Vec.load('lib/d2v_dmm_indo')

print('word2vec loaded')

# words embedding
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] =np.append( model_ug_cbow.wv[w],model_ug_sg.wv[w])
    #embeddings_index[w] = model_ug_cbow.wv[w]

num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('embed')

In [ ]:
model=getModel(num_words, 200, embedding_matrix, 70, traina)

In [ ]:
saveModel(model,"lib/model_dbow-dmm_indo.json")

In [ ]:
history=train(model,lanjut, 'lib/weights_dbow-dmm.hdf5',input_train,target_train,input_val,target_val,ep,bs)

In [ ]:
score('lib/weights_dbow-dmm.hdf5',input_val,target_val)

In [ ]:
plot(history)

## DBOW - DMC

In [ ]:
model_ug_cbow = Doc2Vec.load('lib/d2v_dbow_indo')
model_ug_sg = Doc2Vec.load('lib/d2v_dmc_indo')

print('word2vec loaded')

# words embedding
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] =np.append( model_ug_cbow.wv[w],model_ug_sg.wv[w])
    #embeddings_index[w] = model_ug_cbow.wv[w]

num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('embed')

In [ ]:
model=getModel(num_words, 200, embedding_matrix, 70, traina)

In [ ]:
saveModel(model,"lib/model_dbow-dmc_indo.json")

In [ ]:
history=train(model,lanjut, 'lib/weights_dbow-dmc.hdf5',input_train,target_train,input_val,target_val,ep,bs)

In [ ]:
score('lib/weights_dbow-dmc.hdf5',input_val,target_val)

In [ ]:
plot(history)

## DMM - DMC

In [ ]:
model_ug_cbow = Doc2Vec.load('lib/d2v_dmm_indo')
model_ug_sg = Doc2Vec.load('lib/d2v_dmc_indo')

print('word2vec loaded')

# words embedding
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] =np.append( model_ug_cbow.wv[w],model_ug_sg.wv[w])
    #embeddings_index[w] = model_ug_cbow.wv[w]

num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('embed')

In [ ]:
model=getModel(num_words, 200, embedding_matrix, 70, traina)

In [ ]:
saveModel(model,"lib/model_dmm-dmc_indo.json")

In [ ]:
history=train(model,lanjut, 'lib/weights_dmm-dmc.hdf5',input_train,target_train,input_val,target_val,ep,bs)

In [ ]:
score('lib/weights_dmm-dmc.hdf5',input_val,target_val)

In [ ]:
plot(history)

## DBOW - DMM - DMC

In [ ]:
model_ug_cbow = Doc2Vec.load('lib/d2v_dbow_indo')
model_ug_sg = Doc2Vec.load('lib/d2v_dmm_indo')
model_ug_sg1 = Doc2Vec.load('lib/d2v_dmc_indo')

print('word2vec loaded')

# words embedding
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] =np.append((np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])),model_ug_sg1.wv[w])
    

num_words = 100000
embedding_matrix = np.zeros((num_words, 300))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('embed')

In [ ]:
model=getModel(num_words, 300, embedding_matrix, 70, traina)

In [ ]:
saveModel(model,"lib/model_dbow-dmm-dmc_indo.json")

In [ ]:
history=train(model,lanjut, 'lib/weights_dbow-dmm-dmc.hdf5',input_train,target_train,input_val,target_val,ep,bs)

In [ ]:
score('lib/weights_dbow-dmm-dmc.hdf5',input_val,target_val)

In [ ]:
plot(history)